<a href="https://colab.research.google.com/github/Luca-Wiehe/nerf_segmentation/blob/main/open_nerf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running OpenNeRF

In [1]:
from google.colab import drive
import os

# set path to project folder
# gdrive_path='/content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/nerf_segmentation/' # Luca's Path
gdrive_path='/content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/' # Luis' Path

# mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# navigate to Google Drive folder
os.chdir(gdrive_path)

# check that we are in the right folder
print(sorted(os.listdir()))

Mounted at /content/gdrive
['.git', '.gitignore', '.ipynb_checkpoints', 'README.md', 'data', 'data_loader.ipynb', 'data_loading', 'exported_model', 'gaussian_splatting.ipynb', 'gs_intro.ipynb', 'open_nerf.ipynb', 'outputs', 'visual_data.ipynb']


Folder Structure in Colab:
```
adl4vc  
|_ nerf_segmentation  
    |_ data  
        |_ lerf  
        |_ replica  
|_ opennerf
```



Installing Requirements (ca. 3 min). Note that you need to be connected to a GPU for a correct installation

In [ ]:
%cd /content/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!apt-get install colmap

# NOTE: Nerfstudio is already part of OpenNeRF so skipped here
# Install nerfstudio
# %cd /content/
# !pip install git+https://github.com/nerfstudio-project/nerfstudio.git

%cd /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation

In [ ]:
#!gsutil cp -r gs://cloud-tpu-checkpoints/detection/projects/openseg/colab/exported_model ./

Installing OpenNerfn (includes NerfStudio Installation) in this directory (ca 4. min)

In [ ]:
%cd /content/gdrive/MyDrive/Uni/adl4vc/opennerf
!python -m pip install -e .
!ns-install-cli
%cd /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation
!pip install git+https://github.com/openai/CLIP.git

Check if opennerf is in the options of the next cells outputs, this means the installation should be correct

In [ ]:
!ns-train --help

Now train opennerf on the preprocessed (!) replica dataset

If you do not have it on your machine already, check out the visual_data.ipynb and run the provided function to preprocess

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(7007)"))

https://5zlqju86emc-496ff2e9c6d22116-7007-colab.googleusercontent.com/


In [ ]:
%cd /content
!pip install colab-xterm
%load_ext colabxterm
%env TERM=xterm
from IPython.display import clear_output
clear_output(wait=True)
print("\033[1m" + "Copy and paste the following command into the terminal window that pops up under this cell." + "\033[0m")
print(f"ns-train opennerf --viewer.websocket-port 7007 --viewer.make-share-url True nerfstudio-data --data /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/nerfstudio_/replica_office0 --downscale-factor 4")
print()
%xterm

Copy and paste the following command into the terminal window that pops up under this cell.
ns-train opennerf --viewer.websocket-port 7007 --viewer.make-share-url True nerfstudio-data --data /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/nerfstudio_/replica_office0 --downscale-factor 4



Launching Xterm...

<IPython.core.display.Javascript object>

Training OpenNeRF

In [ ]:
%cd /content
!ns-train opennerf --data /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/nerfstudio_/replica_office0 --output-dir /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/outputs

/content
2024-05-16 16:56:37.103528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 16:56:37.103582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 16:56:37.105234: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-16 16:56:37.113357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 16:56:38.715427: W tensor

In [ ]:
!ns-viewer --viewer.websocket-port 7007 --viewer.make-share-url True --load-config /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/outputs/replica_office0/opennerf/2024-05-15_083414/config.yml
#/content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/outputs/replica_office0/opennerf/2024-05-12_191324/config.yml

2024-05-15 08:39:16.157319: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 08:39:16.157395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 08:39:16.159435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 08:39:16.170694: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 08:39:17.898008: W tensorflow/compiler/tf2

In [ ]:
!ns-export pointcloud --load-config /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/outputs/replica_office0/opennerf/2024-05-15_083414/config.yml --output-dir /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/outputs

2024-05-15 08:44:18.751949: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 08:44:18.751995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 08:44:18.753696: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 08:44:18.762187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 08:44:20.063230: W tensorflow/compiler/tf2

Training Splatfacto to check if NerfStudio works

In [ ]:
!ns-train splatfacto --data /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/nerfstudio_/replica_office0

In [ ]:
!ns-viewer --load-config /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/outputs/replica_office0/splatfacto/2024-05-12_192758/config.yml